In [1]:
# Author : Arthur Prigent
# Email: aprigent@geomar.de

In [2]:
from load_librairies import *
import xscale.signal.fitting as xsf
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

period_str_1 = 1982
period_end_1 = 2017



def is_jja(month):
    return (month >= 6) & (month <= 8)
def is_mjj(month):
    return (month >= 5) & (month <= 7)
def is_amj(month):
    return (month >= 4) & (month <= 6)

def relative_std(a, b):
    return (a - b) / a

def f_test(x, y):
    x = np.array(x)
    y = np.array(y)
    f = np.var(x, ddof=1)/np.var(y, ddof=1) #calculate F test statistic 
    dfn = x.size-1 #define degrees of freedom numerator 
    dfd = y.size-1 #define degrees of freedom denominator 
    p = 1-stats.f.cdf(f, dfn, dfd) #find p-value of F test statistic 
    return f, p

def month_range_day(start=None, periods=None):
    start_date = pd.Timestamp(start).date()
    month_range = pd.date_range(start=start_date, periods=periods, freq='M')
    month_day = month_range.day.values
    month_day[start_date.day < month_day] = start_date.day
    return pd.to_datetime(month_range.year*10000+month_range.month*100+month_day, format='%Y%m%d')
######################
#      TO CHANGE     # 
######################

path_fig = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/figures/new_version/reviews/'
path_data = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/scripts/new_version/to_publish/data/'

In [3]:
nc = xr.open_dataset(
    '/Users/aprigent/Documents/Thesis_GEOMAR/Data/ERA_INTERIM/ERA_interim_uv_1979_2018_1x1.nc')
nc = nc.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
u10_eraint = nc.u10[:,:,:]
u10_eraint = xr.concat([u10_eraint[:, :, 180:], u10_eraint[:, :, :180]], dim='longitude')
u10_eraint.coords['longitude'] = (u10_eraint.coords['longitude'] + 180) % 360 - 180

u10_eraint = u10_eraint.chunk(chunks=None)
u10_eraint = xsf.detrend(u10_eraint,dim='time',type='linear')


u10_eraint_atl4 = Atools.data_sub(u10_eraint,-40,-20,-3,3).mean(dim='longitude').mean(dim='latitude')
u10a_eraint_atl4,_ = Atools.ano_norm_t(u10_eraint_atl4.load())
std_u10a_eraint_atl4_amj = u10a_eraint_atl4.sel(time=is_mjj(
    u10a_eraint_atl4['time.month'])).std(dim='time')

In [4]:
nc = xr.open_dataset(
    '/Users/aprigent/Documents/Thesis_GEOMAR/Data/NCEP-R2/uwnd.10m.mon.mean_NCEPDOE.nc')
nc = nc.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
u10_ncep = nc.uwnd[:,0,:,:]
u10_ncep = xr.concat([u10_ncep[:, :, 96:], u10_ncep[:, :, :96]], dim='lon')
u10_ncep.coords['lon'] = (u10_ncep.coords['lon'] + 180) % 360 - 180

u10_ncep = u10_ncep.chunk(chunks=None)
u10_ncep = xsf.detrend(u10_ncep,dim='time',type='linear')

u10_ncep_atl4 = Atools.data_sub(u10_ncep,-40,-20,-3,3).mean(dim='lon').mean(dim='lat')
u10a_ncep_atl4,_ = Atools.ano_norm_t(u10_ncep_atl4.load())
std_u10a_ncep_atl4_amj = u10a_ncep_atl4.sel(time=is_mjj(
    u10a_ncep_atl4['time.month'])).std(dim='time')


In [5]:
## Load mask ##
nc_mask = xr.open_dataset(
    '/Users/aprigent/Documents/Thesis_GEOMAR/Data/ERA5/ERA_landsea_mask.nc')
lsm = nc_mask.lsm[0,:,:]
lsm = xr.concat([lsm[ :, 720:], lsm[ :, :720]], dim='longitude')
lsm.coords['longitude'] = (lsm.coords['longitude'] + 180) % 360 - 180

## Load data ##
nc = xr.open_dataset(
    '/Users/aprigent/Documents/Thesis_GEOMAR/Data/ERA5/era5_uwnd10m_1982_2019.nc')

## Take the period 1982/01 - 2017/12 ##
nc = nc.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
u10_era5 = nc.u10
u10_era5 = xr.concat([u10_era5[:, :, 720:], u10_era5[:, :, :720]], dim='longitude')
u10_era5.coords['longitude'] = (u10_era5.coords['longitude'] + 180) % 360 - 180

## Apply mask ##
u10_era5 = u10_era5.where(lsm[:,:]==0)

## Detrend data ##
u10_era5_nondetrend = u10_era5
u10_era5 = u10_era5.chunk(chunks=None)
u10_era5 = xsf.detrend(u10_era5,dim='time',type='linear')
u10_era5_atl4 = Atools.data_sub(u10_era5,-40,-20,-3,3).mean(dim='longitude').mean(dim='latitude')
u10a_era5_atl4,_ = Atools.ano_norm_t(u10_era5_atl4.load())
std_u10a_era5_atl4_amj = u10a_era5_atl4.sel(time=is_mjj(
    u10a_era5_atl4['time.month'])).std(dim='time')

In [6]:
obs_u10_std_amj = np.hstack((np.round(std_u10a_ncep_atl4_amj.values,2),
                             np.round(std_u10a_eraint_atl4_amj.values,2),
                             np.round(std_u10a_era5_atl4_amj.values,2)))

In [7]:
print(np.round(obs_u10_std_amj.mean(),2),'+/-',np.round(obs_u10_std_amj.std(),2))

0.66 +/- 0.02


# Observations SST

In [3]:
# 1854-2022
sst_ersst = xr.open_dataset('/Users/aprigent/Documents/Thesis_GEOMAR/Data/ERSSTv5/sst.mnmean.ersstv5.nc') 

sst_ersst = xr.concat([sst_ersst.sst[:, :, 90:], sst_ersst.sst[ :,:, :90]], dim='lon')
sst_ersst.coords['lon'] = (sst_ersst.coords['lon'] + 180) % 360 - 180
sst_ersst_1 = sst_ersst.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
sst_ersst_1 = sst_ersst_1.chunk(chunks=None)
sst_ersst_1 = xsf.detrend(sst_ersst_1,dim='time',type='linear')



In [4]:
sst_ersst_atl3_1 = Atools.data_sub(sst_ersst_1,-20,0,-3,3).mean(dim='lon').mean(dim='lat')
ssta_ersst_atl3_1,_ = Atools.ano_norm_t(sst_ersst_atl3_1.load())
std_ssta_ersst_atl3_jja_1 = ssta_ersst_atl3_1.sel(time=is_jja(
    ssta_ersst_atl3_1['time.month'])).std(dim='time')


# ERA5 

In [5]:
# 1979-2020
## Load mask ##
nc_mask = xr.open_dataset(
    '/Users/aprigent/Documents/Thesis_GEOMAR/Data/ERA5/ERA_landsea_mask.nc')
lsm = nc_mask.lsm[0,:,:]
lsm = xr.concat([lsm[ :, 720:], lsm[ :, :720]], dim='longitude')
lsm.coords['longitude'] = (lsm.coords['longitude'] + 180) % 360 - 180

### Load data ##
#nc = xr.open_dataset(
#    '/Users/aprigent/Documents/Thesis_GEOMAR/Data/ERA5/era5_sst_1982_2019.nc')
#sst_era5_1 = xr.open_dataset('/Users/aprigent/Documents/Thesis_GEOMAR/Data/ERA5/era5.sst.atl.1950.1978.nc')
sst_era5_tmp = xr.open_dataset('/Users/aprigent/Documents/Thesis_GEOMAR/Data/ERA5/era5.sst.atl.1979.2020.nc')
sst_era5 = sst_era5_tmp.sst[:,0,:,:] - 273.15
## Take the period 1982/01 - 2017/12 ##
sst_era5_1 = sst_era5.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
sst_era5_1 = sst_era5_1.chunk(chunks=None)
sst_era5_1 = xsf.detrend(sst_era5_1,dim='time',type='linear')

sst_era5_atl3_1 = Atools.data_sub(sst_era5_1,-20,0,-3,3).mean(dim='longitude').mean(dim='latitude')
ssta_era5_atl3_1,_ = Atools.ano_norm_t(sst_era5_atl3_1.load())
std_ssta_era5_atl3_jja_1 = ssta_era5_atl3_1.sel(time=is_jja(
    ssta_era5_atl3_1['time.month'])).std(dim='time')

# OI-SST

In [6]:
# 1981-2019
## Load the data ##
nc = xr.open_dataset('/Users/aprigent/Documents/Thesis_GEOMAR/Data/OI_SST_v2/oi.sst.v2.monmean.nc')
sst_oiss = nc.sst[:]
sst_oiss = xr.concat([sst_oiss[:, :, 720:], sst_oiss[:, :, :720]], dim='lon')
sst_oiss.coords['lon'] = (sst_oiss.coords['lon'] + 180) % 360 - 180

## Take the period 1982/01-2017/12 ##
sst_oiss_1 = sst_oiss.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
sst_oiss_1 = sst_oiss_1.chunk(chunks=None)
sst_oiss_1 = xsf.detrend(sst_oiss_1,dim='time',type='linear')



In [7]:
sst_oiss_atl3_1 = Atools.data_sub(sst_oiss_1,-20,0,-3,3).mean(dim='lon').mean(dim='lat')
ssta_oiss_atl3_1,_ = Atools.ano_norm_t(sst_oiss_atl3_1.load())
std_ssta_oiss_atl3_jja_1 = ssta_oiss_atl3_1.sel(time=is_jja(
    ssta_oiss_atl3_1['time.month'])).std(dim='time')



# HadI-SST

In [8]:
## Load data ##
nc = xr.open_mfdataset(
    '/Users/aprigent/Documents/Thesis_GEOMAR/Data/HADI_SST/HadISST_sst.nc')
sst_hadi = nc.sst[:]
sst_hadi = sst_hadi.where(sst_hadi>-100)

## take the period 1982/01- 2017/12 ##
sst_hadi_1 = sst_hadi.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
sst_hadi_1 = sst_hadi_1.chunk(chunks=None)
sst_hadi_1 = xsf.detrend(sst_hadi_1,dim='time',type='linear')


In [9]:
sst_hadi_atl3_1 = Atools.data_sub(sst_hadi_1,-20,0,-3,3).mean(dim='longitude').mean(dim='latitude')
ssta_hadi_atl3_1,_ = Atools.ano_norm_t(sst_hadi_atl3_1.load())
std_ssta_hadi_atl3_jja_1 = ssta_hadi_atl3_1.sel(time=is_jja(
    ssta_hadi_atl3_1['time.month'])).std(dim='time')



In [10]:
ens_obs_jja_1 = [std_ssta_ersst_atl3_jja_1.values,std_ssta_oiss_atl3_jja_1.values,
           std_ssta_hadi_atl3_jja_1.values,std_ssta_era5_atl3_jja_1.values]



In [11]:
print('OBS ens mean 1982-2017 = ',np.round(np.mean(ens_obs_jja_1),2),'+/-',np.round(np.std(ens_obs_jja_1),2))


OBS ens mean 1982-2017 =  0.54 +/- 0.02


In [12]:
ens_obs_jja_1

[array(0.52150044), array(0.52388235), array(0.52271364), array(0.57929921)]